In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as f
from pathlib import Path
import numpy as np
import torchvision
from torch.utils.data import DataLoader, Dataset
import os
from torchsummary import torchsummary
from PIL import Image

In [2]:
ORIGINAL_IMAGE = 'E:/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = 'E:/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'
POSITIVE_RATIO = 0.1
BATCH_SIZE = 8

In [ ]:
ORIGINAL_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'
POSITIVE_RATIO = 0.1
BATCH_SIZE = 8

## CosineEmbeddingLoss

In [4]:
class NDI_dataset(Dataset):
    def __init__(self, positive_ratio=0.5, times=10):
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
        self.origins, self.targets, self.label = [], [], []
        for original_image in original_images:
            self.origins.append(torchvision.io.read_image(str(original_image)))
            self.targets.append(torchvision.io.read_image(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg'))))
            self.label.append(torch.tensor(1))
        for _ in range(times):
            for original_image in original_images:
                self.origins.append(torchvision.io.read_image(str(original_image)))
                self.targets.append(torchvision.io.read_image(str(np.random.choice(target_images))))
                self.label.append(torch.tensor(-1))

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx], self.label[idx]

    def __len__(self):
        return len(self.origins)

In [5]:
train_iter = DataLoader(NDI_dataset(POSITIVE_RATIO), BATCH_SIZE, shuffle=True)

In [6]:
model = torchvision.models.resnet18(pretrained=False)

In [7]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(512, 256)

In [8]:
torchsummary.summary(model, input_size=(1, 200, 200), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           3,136
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1, 64,

In [9]:
device = torch.device('cuda')
lr = 0.001
EPOCHS = 20

loss = nn.CosineEmbeddingLoss(margin=0.3)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.to(device)
train_iter = DataLoader(NDI_dataset(POSITIVE_RATIO), BATCH_SIZE, shuffle=True)
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for origin, target, label in train_iter:
        optimizer.zero_grad()
        origin, target = origin.float(), target.float()
        origin, target, label = origin.to(device), target.to(device), label.to(device)
        origin_feature = model(origin)
        target_feature = model(target)
        l = loss(origin_feature, target_feature, label)
        l.backward()
        total_loss += l
    print(f'Epoch: {epoch + 1}, Ave. Loss: {total_loss / len(train_iter):.4f}')


Epoch: 1, Ave. Loss: 0.5940
Epoch: 2, Ave. Loss: 0.5952
Epoch: 3, Ave. Loss: 0.5944
Epoch: 4, Ave. Loss: 0.5929
Epoch: 5, Ave. Loss: 0.5938
Epoch: 6, Ave. Loss: 0.5953
Epoch: 7, Ave. Loss: 0.5931
Epoch: 8, Ave. Loss: 0.5945
Epoch: 9, Ave. Loss: 0.5908
Epoch: 10, Ave. Loss: 0.5949
Epoch: 11, Ave. Loss: 0.5910
Epoch: 12, Ave. Loss: 0.5936
Epoch: 13, Ave. Loss: 0.5947
Epoch: 14, Ave. Loss: 0.5938
Epoch: 15, Ave. Loss: 0.5946
Epoch: 16, Ave. Loss: 0.5942
Epoch: 17, Ave. Loss: 0.5944
Epoch: 18, Ave. Loss: 0.5940
Epoch: 19, Ave. Loss: 0.5937
Epoch: 20, Ave. Loss: 0.5939


In [10]:
from PIL import Image

model.to('cpu')
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
original_image_name = np.random.choice(original_images)
original_image = torchvision.io.read_image(str(original_image_name)).unsqueeze_(0).float()
target_image = torchvision.io.read_image(str(Path.joinpath(Path(TARGET_IMAGE), original_image_name.name.split('_')[0] + '.jpg'))).unsqueeze_(0).float()
print(nn.CosineSimilarity()(model(original_image), model(target_image)))

tensor([0.9949], grad_fn=<DivBackward0>)


In [11]:
model.to('cpu')
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
original_image_name = np.random.choice(original_images)
target_image_name = np.random.choice(target_images)
original_image = torchvision.io.read_image(str(original_image_name)).unsqueeze_(0).float()
target_image = torchvision.io.read_image(str(target_image_name)).unsqueeze_(0).float()
print(nn.CosineSimilarity()(model(original_image), model(target_image)))

tensor([0.9946], grad_fn=<DivBackward0>)


In [129]:
dimensions = 512
a = torch.rand((1, dimensions))
b = torch.rand((1, dimensions))
nn.CosineSimilarity()(a, b)

tensor([0.7585])

## Binary Classification

In [7]:
a = torch.rand((8, 512))
b = torch.rand((8, 512))
torch.cat([a, b], dim=).size()

torch.Size([16, 512])

In [11]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.backbone = torchvision.models.resnet18(pretrained=False)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.backbone.fc = nn.Linear(512, 256)
        self.classifier = nn.Linear(512, 2)
    def forward(self, origin, target):
        origin_feature = f.relu(self.backbone(origin))
        target_feature = f.relu(self.backbone(target))
        return self.classifier(torch.cat([origin_feature, target_feature], dim=-1))

In [12]:
class NDI_dataset_classifier(Dataset):
    def __init__(self, positive_ratio=0.5, times=10):
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
        self.origins, self.targets, self.label = [], [], []
        for original_image in original_images:
            self.origins.append(torchvision.io.read_image(str(original_image)))
            self.targets.append(torchvision.io.read_image(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg'))))
            self.label.append(torch.tensor(1, dtype=torch.long))
        for _ in range(times):
            for original_image in original_images:
                self.origins.append(torchvision.io.read_image(str(original_image)))
                self.targets.append(torchvision.io.read_image(str(np.random.choice(target_images))))
                self.label.append(torch.tensor(0, dtype=torch.long))

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx], self.label[idx]

    def __len__(self):
        return len(self.origins)

In [16]:
model = ClassificationModel()
torchsummary.summary(model, input_size=[(1, 200, 200), (1, 200, 200)], device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           3,136
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1, 64,

In [ ]:
device = torch.device('cuda')
lr = 0.001
EPOCHS = 20

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.to(device)
train_iter = DataLoader(NDI_dataset(POSITIVE_RATIO), BATCH_SIZE, shuffle=True)
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for origin, target, label in train_iter:
        optimizer.zero_grad()
        origin, target = origin.float(), target.float()
        origin, target, label = origin.to(device), target.to(device), label.to(device)
        origin_feature = model(origin)
        target_feature = model(target)
        l = loss(origin_feature, target_feature, label)
        l.backward()
        total_loss += l
    print(f'Epoch: {epoch + 1}, Ave. Loss: {total_loss / len(train_iter):.4f}')

In [17]:
x = torch.rand((1, 2))

In [20]:
torch.argmax(x).type(x.dtype).sum()

tensor(0.)

In [23]:
torch.rand(1)

tensor([0.6341])

In [28]:
mnist_dataset = torchvision.datasets.FashionMNIST(root='../data', train=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]), download=True)
mnist_loader = torch.utils.data.DataLoader(mnist_dataset, batch_size=32, shuffle=True)
for x, y in mnist_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])


In [33]:
torch.max(x[0], dim=2)

torch.return_types.max(
values=tensor([[0.7490, 0.9333, 0.8863, 0.8824, 0.8784, 0.8196, 0.8078, 0.8510, 0.8941,
         0.9647, 0.9176, 1.0000, 0.8588, 0.9216, 0.9294, 0.9882, 0.9882, 0.9882,
         0.9843, 0.9765, 0.9451, 0.9843, 0.9843, 0.9412, 0.8627, 0.9020, 0.8431,
         0.6902]]),
indices=tensor([[13, 15, 10, 16,  6, 18, 18,  7,  7, 19, 19, 19, 20,  9,  9,  6, 18, 18,
         18, 18, 18, 21, 21,  8, 21, 19, 19, 22]]))

In [25]:
class NDIDatasetContrastiveLearning(Dataset):
    def __init__(self):
        super(NDIDatasetContrastiveLearning, self).__init__()
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        origins, targets = [], []
        to_tensor_func = torchvision.transforms.ToTensor()
        for original_image in original_images:
            origins.append(to_tensor_func(Image.open(str(original_image))))
            targets.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))))
        random_index = np.random.permutation(len(origins))
        self.origins, self.targets = [], []
        for index in random_index:
            self.origins.append(origins[index])
            self.targets.append(targets[index])

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx]

    def __len__(self):
        return len(self.origins)

In [108]:
from moco_model import MoCo


model = MoCo(torchvision.models.resnet18, dim=128, K=48)
device = torch.device('cuda:0')
criterion = nn.CrossEntropyLoss().cuda(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)

In [109]:
def train_moco(net, criterion, optimizer, epochs, device):
    train_iter = DataLoader(NDIDatasetContrastiveLearning(), batch_size=8, shuffle=True, drop_last=True)
    model.cuda(device)
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for origin, target in train_iter:
            origin, target = origin.cuda(device), target.cuda(device)
            output, labels = net(origin, target)
            loss = criterion(output, labels)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss {total_loss / len(train_iter)}')


In [110]:
# 把K设置成48还算成功，在queue不大的情况下

train_moco(model, criterion, optimizer, 50, device)

Epoch 1, Loss 2.5478965307144485
Epoch 2, Loss 3.8372589349746704
Epoch 3, Loss 3.6488163073857627
Epoch 4, Loss 3.4865161577860513
Epoch 5, Loss 3.177033305168152
Epoch 6, Loss 3.2050740321477256
Epoch 7, Loss 3.020328919092814
Epoch 8, Loss 3.082185745239258
Epoch 9, Loss 2.824070692062378
Epoch 10, Loss 2.7290110190709433
Epoch 11, Loss 2.962373971939087
Epoch 12, Loss 2.529419183731079
Epoch 13, Loss 2.5628720919291177
Epoch 14, Loss 2.4155572255452475
Epoch 15, Loss 2.3799455563227334
Epoch 16, Loss 2.3157723347345986
Epoch 17, Loss 2.179949184258779
Epoch 18, Loss 2.2039785782496133
Epoch 19, Loss 2.0526958306630454
Epoch 20, Loss 2.06346728404363
Epoch 21, Loss 2.0119340817133584
Epoch 22, Loss 1.9778805375099182
Epoch 23, Loss 1.8880394498507183
Epoch 24, Loss 1.9622933467229207
Epoch 25, Loss 1.756432831287384
Epoch 26, Loss 1.8690356214841206
Epoch 27, Loss 1.8484575748443604
Epoch 28, Loss 1.7700023452440898
Epoch 29, Loss 1.6966349085172017
Epoch 30, Loss 1.7569868365923564

In [95]:
# 尝试把K设置成56

model = MoCo(torchvision.models.resnet18, dim=128, K=56)
device = torch.device('cuda:0')
criterion = nn.CrossEntropyLoss().cuda(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)
train_moco(model, criterion, optimizer, 50, device)

Epoch 1, Loss 2.673976675757634
Epoch 2, Loss 3.931485931078593
Epoch 3, Loss 3.8541926542917886
Epoch 4, Loss 3.6459306478500366
Epoch 5, Loss 3.407223383585612
Epoch 6, Loss 3.105238596598307
Epoch 7, Loss 3.1296446720759072
Epoch 8, Loss 3.1514963706334433
Epoch 9, Loss 3.053861141204834
Epoch 10, Loss 2.6810205380121865
Epoch 11, Loss 2.705527981122335
Epoch 12, Loss 2.632307688395182
Epoch 13, Loss 2.4753270546595254
Epoch 14, Loss 2.3976933558781943
Epoch 15, Loss 2.333985129992167
Epoch 16, Loss 2.2366109689076743
Epoch 17, Loss 2.1844319701194763
Epoch 18, Loss 2.2525221506754556
Epoch 19, Loss 2.1583401759465537
Epoch 20, Loss 2.0467366774876914
Epoch 21, Loss 2.0968031088511148
Epoch 22, Loss 2.081486225128174
Epoch 23, Loss 2.047618548075358
Epoch 24, Loss 1.9676475127538045
Epoch 25, Loss 1.9295819004376729
Epoch 26, Loss 1.8663320342699687
Epoch 27, Loss 1.8291430870691936
Epoch 28, Loss 1.789828399817149
Epoch 29, Loss 1.7487680315971375
Epoch 30, Loss 1.7118704319000244


In [41]:
def image_pair_matching(net, original_image, matching_image):
    net.cpu()
    net.eval()
    q = net.encoder_q(original_image)
    q = f.normalize(q, dim=1)
    k = net.encoder_k(matching_image)
    k = f.normalize(k, dim=1)
    logits = torch.mm(q, k.T)
    return logits

In [93]:
# 取总共大小小于K的queue来试，成功

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
original_image = np.random.choice(original_images)
original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_images_candidates = np.random.choice(target_images, 40)
target_tensor = []
for i in range(20):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))).unsqueeze(0))
for i in range(20, 40):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)

In [94]:
image_pair_matching(model, original_tensor, target_tensor)

tensor([[ 0.2281,  0.1505,  0.0759,  0.3091,  0.0944,  0.3091, -0.0131,  0.1535,
          0.0576,  0.1025,  0.0797,  0.1233,  0.1377,  0.0640,  0.1535,  0.0891,
         -0.0098,  0.0383,  0.2141, -0.0383,  0.3091,  0.0969,  0.0739,  0.0576,
          0.1209,  0.1377,  0.0891,  0.1424,  0.0175,  0.1025,  0.1732,  0.0883,
          0.0811,  0.0613,  0.1233, -0.0235,  0.1025,  0.0883,  0.0738,  0.2119,
          0.0383]], grad_fn=<MmBackward>)

In [113]:
# 取所有比对图片

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
original_image = np.random.choice(original_images)
original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_tensor = []
for i in range(1, 55):
    target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), f'{i}.jpg')))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
print(str(original_image.name).split('_')[0])

21


In [114]:
logits = image_pair_matching(model, original_tensor, target_tensor)
print(logits)
print(torch.argmax(logits))

tensor([[ 1.4565e-01,  1.1097e-01,  3.9617e-02,  1.2304e-01,  1.0800e-01,
          1.1116e-01,  1.9075e-01,  1.6977e-01,  7.8928e-02,  2.5626e-01,
          9.6446e-02,  1.6837e-01,  8.6016e-02,  1.0304e-01,  9.4899e-02,
          3.0067e-02,  8.0622e-02,  1.3401e-01,  8.8296e-02, -1.2011e-03,
          4.0938e-01,  5.5814e-02,  1.7832e-01,  6.8754e-02,  3.9356e-01,
          3.9356e-01,  2.4748e-01, -9.8140e-02, -9.8763e-02,  1.2288e-01,
          1.8330e-01,  1.7046e-01,  7.5117e-02,  1.4688e-01,  1.3979e-01,
          1.2716e-04,  1.2593e-01,  7.9922e-02,  3.0441e-01,  3.9980e-02,
          2.0772e-02,  1.7252e-01,  1.3889e-01,  3.5146e-03,  3.6427e-02,
          1.8339e-01,  8.3577e-02,  5.3283e-02,  1.7072e-01,  1.6657e-01,
          1.9643e-01,  5.9414e-02,  5.9414e-02,  1.3753e-01]],
       grad_fn=<MmBackward>)
tensor(20)
